<a href="https://colab.research.google.com/github/mohamedamrfarouk/arabic_dialect_classifier/blob/main/data_fetching_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import the needed libraries for fetching the data

In [2]:
import requests
import numpy as np
import pandas as pd
import json
import re


# load the csv file in the data frame

[the dataset](https://drive.google.com/file/d/1Q3wDYQk6UQTtRu5L4167Xs9k6aC4Dra-/view?usp=sharing)

In [3]:
df = pd.read_csv("/content/dialect_dataset.csv")
df.dropna()
df['id']=df['id'].apply(str)

# get the json 

In [4]:
url = 'https://recruitment.aimtechnologies.co/ai-tasks'
data = {}
id_list = df.id.tolist()
id_list = [str(x) for x in id_list]

for i in range(int(len(df)/1000)):
  payload = id_list[i*1000:(i+1)*1000]
  r = requests.post(url, data=json.dumps(payload)) 
  data.update( r.json() )


payload = id_list[len(data):len(df)]
r = requests.post(url, data=json.dumps(payload)) 
data.update( r.json() )

# convert the dictionary to dataframe

In [5]:
dataframe = pd.DataFrame.from_dict(data, orient='index', columns=['text'])
dataframe = dataframe.reset_index()
dataframe = dataframe.rename(columns={"index":"id"})
dataframe.head(5)

,id,text
0,1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


# merge the two dataframes on ID

In [6]:
result_dataframe = pd.merge(df, dataframe, on='id',  how='left')
result_dataframe.head()

,id,dialect,text
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


# save the dataframe after merging in a CSV file

In [10]:
result_dataframe.to_csv("/content/dialect_dataset_with_text.csv",index=False)